In [8]:
# Object Graph Feature Extraction for Rotated Bounding Boxes

import json
import numpy as np
import os

# Function to load JSON data
def load_json(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

# Function to calculate area of rotated bounding box
def calculate_rotated_bbox_area(bbox):
    _, _, width, height, _, _ = bbox
    return width * height

# Function to build adjacency and degree matrices
def build_matrices(json_file_path, num_classes=20, image_size=(255, 255)):
    data = load_json(json_file_path)
    num_objects = len(data)

    # Limit the number of objects to the predefined number of classes (20)
    num_objects = min(num_objects, num_classes)

    # Initialize adjacency and degree matrices
    adjacency_matrix = np.zeros((num_classes, num_classes), dtype=float)
    degree_matrix = np.zeros((num_classes, num_classes), dtype=float)
    
    # Create a label mapping based on the JSON data without removing duplicates
    label_mapping = {f"obj{i}": i for i in range(num_objects)}
    for i in range(num_objects, num_classes):
        label_mapping[f"nothing{i}"] = i

    # Iterate through all objects and populate the matrices
    for i in range(num_objects):
        obj1 = data[i]
        class_idx_1 = label_mapping[f"obj{i}"]

        # Calculate degree contribution (normalized area * probability)
        bbox_area = calculate_rotated_bbox_area(obj1['bbox'])
        normalized_area = bbox_area / (image_size[0] * image_size[1])  # Normalize area by image size
        probability = abs(obj1['bbox'][-1])  # Example to get probability
        degree_matrix[class_idx_1, class_idx_1] += (normalized_area * probability)

        # Calculate adjacency relationships between every pair of objects
        for j in range(num_objects):
            if i == j:
                continue
            obj2 = data[j]
            class_idx_2 = label_mapping[f"obj{j}"]

            # Relative distance can be calculated here between object pairs
            x1, y1, _, _, _, _ = obj1['bbox']
            x2, y2, _, _, _, _ = obj2['bbox']
            distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

            # Arbitrary calculation for adjacency weight based on distance
            if distance > 0:
                weight = 1.0 / distance
            else:
                weight = 0  # Distance should not be zero unless objects overlap perfectly
            
            adjacency_matrix[class_idx_1, class_idx_2] += weight

    # Set diagonal of adjacency matrix to zero
    np.fill_diagonal(adjacency_matrix, 0)
    return adjacency_matrix, degree_matrix

# Example usage
json_path = "./det_json/airport_2.json"
adjacency_matrix, degree_matrix = build_matrices(json_path)

print("Adjacency Matrix:")
print(adjacency_matrix)

print("\nDegree Matrix:")
print(degree_matrix)


Adjacency Matrix:
[[0.00000000e+00 3.72388778e-02 2.90576126e-02 2.07760912e-02
  1.63797609e-02 1.20954249e-02 1.40450027e-02 8.49080359e-03
  5.67504931e-03 1.34478212e-02 1.64340507e-02 2.09790308e-02
  1.21020227e-02 2.90649125e-02 1.40450027e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.72388778e-02 0.00000000e+00 2.14231845e-02 1.39059731e-02
  1.15837312e-02 1.69885204e-02 2.18606081e-02 8.83941312e-03
  6.60008010e-03 1.16659264e-02 1.16064384e-02 1.39415735e-02
  1.69915801e-02 2.15562505e-02 2.18606081e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.90576126e-02 2.14231845e-02 0.00000000e+00 2.83247576e-02
  1.93277401e-02 1.23008268e-02 1.10508635e-02 6.58190734e-03
  5.11171844e-03 9.37690262e-03 1.93369512e-02 2.78377359e-02
  1.23188640e-02 2.03248857e+00 1.10508635e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.07760912e-02 1.39059731e-02 2.83247576e-02 0.